In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/poetry/Kanye_West.txt
/kaggle/input/poetry/johnny-cash.txt
/kaggle/input/poetry/kanye-west.txt
/kaggle/input/poetry/bruno-mars.txt
/kaggle/input/poetry/dickinson.txt
/kaggle/input/poetry/amy-winehouse.txt
/kaggle/input/poetry/blink-182.txt
/kaggle/input/poetry/paul-simon.txt
/kaggle/input/poetry/patti-smith.txt
/kaggle/input/poetry/bieber.txt
/kaggle/input/poetry/disney.txt
/kaggle/input/poetry/jimi-hendrix.txt
/kaggle/input/poetry/lin-manuel-miranda.txt
/kaggle/input/poetry/adele.txt
/kaggle/input/poetry/dj-khaled.txt
/kaggle/input/poetry/beatles.txt
/kaggle/input/poetry/r-kelly.txt
/kaggle/input/poetry/lady-gaga.txt
/kaggle/input/poetry/radiohead.txt
/kaggle/input/poetry/britney-spears.txt
/kaggle/input/poetry/alicia-keys.txt
/kaggle/input/poetry/rihanna.txt
/kaggle/input/poetry/joni-mitchell.txt
/kaggle/input/poetry/dolly-parton.txt
/kaggle/input/poetry/drake.txt
/kaggle/input/poetry/Lil_Wayne.txt
/kaggle/input/poetry/notorious_big.txt
/kaggle/input/poetry/eminem.txt
/

In [2]:
import tensorflow as tf
import keras

# GOAL - create a bot to generate song lyrics using RNN

In [3]:
artist = '/kaggle/input/poetry/Kanye_West.txt'
char_window = 20
step = 3

In [4]:
with open(artist) as f:
    text = f.read().lower()

In [5]:
import re

text = re.sub('[^a-z \n]','',text)

In [6]:
chars = sorted(list(set(text)))
chars = chars[:28]
char_to_index = dict((c, i) for i, c in enumerate(chars))
index_to_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
inputs = []
next_chars = []

for i in range(0,len(text)-char_window,step):
    inputs.append(text[i:i+char_window])
    next_chars.append(text[i+char_window])
        
pairs = zip(inputs,next_chars)
list(pairs)[:100]

[('eighteen years eight', 'e'),
 ('hteen years eighteen', ' '),
 ('en years eighteen ye', 'a'),
 ('years eighteen years', '\n'),
 ('rs eighteen years\nsh', 'e'),
 ('eighteen years\nshe g', 'o'),
 ('hteen years\nshe got ', 'o'),
 ('en years\nshe got one', ' '),
 ('years\nshe got one of', ' '),
 ('rs\nshe got one of yo', 'u'),
 ('she got one of your ', 'k'),
 (' got one of your kid', 's'),
 ('t one of your kids g', 'o'),
 ('ne of your kids got ', 'y'),
 ('of your kids got you', ' '),
 ('your kids got you fo', 'r'),
 ('r kids got you for e', 'i'),
 ('ids got you for eigh', 't'),
 (' got you for eightee', 'n'),
 ('t you for eighteen y', 'e'),
 ('ou for eighteen year', 's'),
 ('for eighteen years\ni', ' '),
 (' eighteen years\ni kn', 'o'),
 ('ghteen years\ni know ', 's'),
 ('een years\ni know som', 'e'),
 (' years\ni know somebo', 'd'),
 ('ars\ni know somebody ', 'p'),
 ('\ni know somebody pay', 'i'),
 ('know somebody paying', ' '),
 ('w somebody paying ch', 'i'),
 ('omebody paying child', 

In [8]:
X = np.zeros((len(inputs),char_window),dtype=np.float32)

for i in range(len(inputs)):
    for j in range(char_window):
        X[i,j] = char_to_index[inputs[i][j]]
        
X

array([[ 6., 10.,  8., ...,  8.,  9., 21.],
       [ 9., 21.,  6., ...,  6.,  6., 15.],
       [ 6., 15.,  1., ...,  1., 26.,  6.],
       ...,
       [14., 17., 16., ...,  1., 21.,  9.],
       [19., 21.,  2., ...,  2., 21.,  1.],
       [15., 21.,  1., ..., 26., 16., 22.]], dtype=float32)

In [9]:
from keras.utils.np_utils import to_categorical

y = np.zeros(len(next_chars),dtype=np.float32)

for i in range(len(next_chars)):
    y[i] = char_to_index[next_chars[i]]

y = to_categorical(y)

y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, LSTM, SimpleRNN, Dense, Dropout

model = Sequential()

model.add(Embedding(28,64,input_length=char_window))
model.add(LSTM(256,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(28,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

2022-03-14 01:45:24.519186: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 64)            1792      
_________________________________________________________________
lstm (LSTM)                  (None, 20, 256)           328704    
_________________________________________________________________
dropout (Dropout)            (None, 20, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 28)                7196      
Total params: 863,004
Trainable params: 863,004
Non-trainable params: 0
__________________________________________________

In [11]:
model.fit(X,y,epochs=5,batch_size=20)

Epoch 1/5


2022-03-14 01:45:25.298162: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


4183/4183 [==============================] - 324s 76ms/step - loss: 2.2201 - accuracy: 0.3423
Epoch 2/5
4183/4183 [==============================] - 327s 78ms/step - loss: 1.7856 - accuracy: 0.4502
Epoch 3/5
4183/4183 [==============================] - 324s 77ms/step - loss: 1.6511 - accuracy: 0.4829
Epoch 4/5
4183/4183 [==============================] - 327s 78ms/step - loss: 1.5623 - accuracy: 0.5082
Epoch 5/5
4183/4183 [==============================] - 327s 78ms/step - loss: 1.4879 - accuracy: 0.5276


In [12]:
model.save('kanye')

2022-03-14 02:12:55.738195: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [13]:
from keras.preprocessing.sequence import pad_sequences

def generate_line(model,context):
    line = context
    next_char = ''
    counter = 0
    while counter<100 and next_char != '\n':
        X = [[char_to_index[line[i]] for i in range(0,len(line))]]
        X = pad_sequences(X,maxlen=char_window)
        y = model.predict(X)
        next_char = index_to_char[np.argmax(y)]
        counter+=1
        line+=next_char
    return line

In [14]:
generate_line(model,'one good girl')

'one good girl i said i want to say i want to say i want to say i want to say i want to say i want to say i want t'

In [15]:
generate_line(model,'i had a dream')

'i had a dreams\n'

In [16]:
generate_line(model,'i need you')

'i need you can start the straight and i say i want to say i want to say i want to say i want to say i want to '

## We are going to try using temperature now

In [17]:
def softmax(z):
    return np.exp(z)/sum(np.exp(z))

def temp_scale(conditional_probability, temperature=1.0):
    conditional_probability = np.asarray(conditional_probability).astype("float64")
    conditional_probability = np.log(conditional_probability) / temperature
    reweighted_conditional_probability = softmax(conditional_probability)
    probas = np.random.multinomial(1, reweighted_conditional_probability, 1)
    return np.argmax(probas)

In [18]:
def generate_line_2(model,context,temp):
    line = context
    next_char = ''
    counter = 0
    while counter<100 and next_char != '\n':
        X = [[char_to_index[line[i]] for i in range(0,len(line))]]
        X = pad_sequences(X,maxlen=char_window)
        y = model.predict(X)
        next_char = index_to_char[temp_scale(y[0,:],temp)]
        counter+=1
        line+=next_char
    return line

In [19]:
temps = [.01, 0.2, 0.5, 1.0, 1.2, 2]

for temp in temps:
    print(generate_line_2(model,'i had a dream i could',temp))

i had a dream i could got a say i want to say i want to say i want to say i want to say i want to say i want to say i wan
i had a dream i could got a still got the straight i want to the streets was the straight now

i had a dream i could about a plane

i had a dream i could buy a song courle it

i had a dream i could never even going up to keep pess on this beverein yeah up the chocuby doin with from time

i had a dream i couldnt llapieaases it tpastrssekcteuning



In [20]:
for temp in temps:
    print(generate_line_2(model,'one good girl',temp))

one good girl i said i want to say i want to say i want to say i want to say i want to say i want to say i want t
one good girl i know what i want to get the stripper that i said i want to the straight and what you aint gotta s
one good girl hey so the strip i will to take you made me

one good girl all your spole

one good girlficred leaving craqks

one good girl enteabdpeid nigga



In [21]:
for temp in temps:
    print(generate_line_2(model,'i need you',temp))

i need you can start the straight and i say i want to say i want to say i want to say i want to say i want to 
i need you aint gotta the straight i can something to the churts and they say the money got a minute the club

i need you aint know she gotta you blaze

i need you i just was ball

i need you know im you been woway him

i need yous fightongout

